In [1]:
import numpy as np
file = "D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_clips_no_zip.npy"
X = np.load(file, allow_pickle=True)
X = np.reshape(X, (X.shape[0], X.shape[1],X.shape[2]*X.shape[3]))
X = X/np.max(X)
#X=np.abs(X)

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
file = "D:\\tuc\\exam10\\Thesis\\Dataset\\y.npy"
Y = np.load(file, allow_pickle=True)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2, random_state = 42)

In [3]:
X_train = X_train.reshape(858, 3198, 186,1)
y_train = y_train.reshape(858,1)
X_test = X_test.reshape(215, 3198, 186,1)
y_test = y_test.reshape(215,1)

In [4]:
print(X_train.shape[1:])

(3198, 186, 1)


In [9]:
del X

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras import utils
from keras.layers import CuDNNLSTM

nNeurons = 32
output_layers = 1
# creating the NN model
model = Sequential()
model.add( LSTM( nNeurons,activation = 'relu' ,return_sequences=True,  input_shape = (3198, 186) ) )
model.add( Dropout(0.2))
model.add( LSTM( nNeurons ,activation = 'relu') )
model.add( Dropout(0.2))
model.add(Flatten())
model.add( Dense( output_layers, activation = 'softmax' ) )

opt = tf.keras.optimizers.Adam(learning_rate =1e-3, decay=1e-5)
model.compile( loss = 'mse',  optimizer = opt, metrics = ['accuracy'] )
model.summary()

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-biggeer.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode = 'min')
#callbacks_list = [checkpoint]

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3198, 32)          28032     
                                                                 
 dropout_2 (Dropout)         (None, 3198, 32)          0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 36,385
Trainable params: 36,385
Non-trai

In [11]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
h=model.fit(X_train, y_train, epochs = 3, batch_size=32) #callbacks=callbacks_list)
#model.save('cmuclassifier.model')

#y_pred = model.predict(X_test)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [14]:
y_pred = model.predict(X_test)
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = []
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))


a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [9]:
#Converting predictions to label
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = []
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))


a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=100, batch_size=8)


plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

Accuracy is: 100.0


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


In [3]:
model = Sequential()
model.add(LSTM(16, input_shape=(x_train.shape[1:]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(16, activation='tanh'))
model.add(Dropout(0.1))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 16)            2880      
                                                                 
 dropout (Dropout)           (None, 28, 16)            0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                1

In [4]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=3,
          validation_data=(x_test, y_test))

Epoch 1/3
1875/1875 [==============================] - 23s 10ms/step - loss: 1.1201 - accuracy: 0.6109 - val_loss: 0.4455 - val_accuracy: 0.8755
Epoch 2/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.5410 - accuracy: 0.8378 - val_loss: 0.2522 - val_accuracy: 0.9310
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.4005 - accuracy: 0.8856 - val_loss: 0.1965 - val_accuracy: 0.9430
